In [14]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
#print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [15]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [16]:
# !pip install plot_model
# !pip install keras
# !pip install --upgrade tensorflow
# !pip install --upgrade tensorflow-gpu

#1. Import Libraries

In [17]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.utils import to_categorical

import tensorflow as tf
print("You are using TensorFlow version", tf.__version__)
if len(tf.config.list_physical_devices('GPU')) > 0:
  print("You have a GPU enabled.")
else:
  print("Enable a GPU before running this notebook.")
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential  # initial NN
from tensorflow.keras.layers import Input, Dense, Activation, Dropout, InputLayer, BatchNormalization # construct each layer
from tensorflow.keras.layers import Conv2D # swipe across the image by 1
from tensorflow.keras.layers import MaxPool2D, GlobalMaxPool2D # swipe across by pool size
from tensorflow.keras.layers import Flatten, GlobalAveragePooling2D

from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau,LearningRateScheduler, TensorBoard, EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy, binary_crossentropy, mean_squared_error,sparse_categorical_crossentropy

import pandas as pd
import numpy as np
from random import shuffle
import os
import cv2
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg 
%matplotlib inline
from tqdm.notebook import trange,tqdm
from IPython.display import Image, display, Markdown, clear_output

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn import svm, metrics
from sklearn.decomposition import PCA
from skimage import io

In [18]:
import cv2
# from keras import layers
# from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
# from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
# from keras.models import Model
# from keras.preprocessing import image
# from keras.utils import layer_utils
# from keras.utils.data_utils import get_file
# from keras.applications.imagenet_utils import preprocess_input
# import pydot
# from IPython.display import SVG
# from keras.utils.vis_utils import model_to_dot
# #from keras.utils import plot_model
# #from keras.utils import to_categorical
# import matplotlib.pyplot as plt 
from random import shuffle
# np.set_printoptions(suppress=True)
# np.set_printoptions(threshold=np.inf)

In [19]:
tf.config.list_physical_devices()

In [20]:
# from google.colab import drive
# drive.mount('/content/drive')

In [21]:
import zipfile

with zipfile.ZipFile('/kaggle/input/dogs-vs-cats-redux-kernels-edition/train.zip','r') as z:
    z.extractall('.')
    
with zipfile.ZipFile('/kaggle/input/dogs-vs-cats-redux-kernels-edition/test.zip','r') as z:
    z.extractall('.')


In [22]:
# !pwd
# !ls -l

#2 EDA

In [23]:
# Create image 
IMG_SIZE = 100
X_Train_orig = []
Y_Train_orig = []
for i in os.listdir('/kaggle/working/train/'):
    label = i.split('.')[-3]
    if label == 'cat':
        label = 0
    elif label == 'dog':
        label = 1
    img = cv2.imread('/kaggle/working/train/'+i, cv2.IMREAD_COLOR)
    img = cv2.resize(img,(IMG_SIZE,IMG_SIZE), interpolation=cv2.INTER_CUBIC)
    #img = img / 255
    X_Train_orig.append([np.array(img), np.array(label)])

np.save('Training_Data.npy', X_Train_orig)

In [24]:
shuffle(X_Train_orig)

In [25]:
X = np.array([i[0] for i in X_Train_orig]).reshape(-1,IMG_SIZE,IMG_SIZE,3)
y = np.array([i[1] for i in X_Train_orig])

In [26]:
#print(X)

In [27]:
#print(y)

In [28]:
print("Shape of X is: ",X.shape)
print("Shape of y is: ",y.shape)

In [29]:
# Spilt Training and test data
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.20, stratify=y)

In [30]:
print('Shape of X_train is :', X_train.shape)
print('Shape of y_train is :', y_train.shape)
print('Shape of X_val is :', X_val.shape)
print('Shape of y_val is :', y_val.shape)

In [31]:
print(pd.Series(y_train).value_counts(1))
print(pd.Series(y_val).value_counts(1))

In [32]:
# X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.25, stratify=Y_train)

In [33]:
# print('Shape of X_train is :', X_train.shape)
# print('Shape of Y_train is :', Y_train.shape)
# print('Shape of X_val is :', X_val.shape)
# print('Shape of Y_val is :', Y_val.shape)

In [34]:
# print(pd.Series(Y_train).value_counts(1))
# print(pd.Series(Y_val).value_counts(1))

In [35]:
import matplotlib.pyplot as plt 
plt.figure(figsize=(20,20))   # to fix a shape for each image print
for i in range(50):          # using a for loop to display a number of images
    plt.subplot(5, 10, i+1) # we need to use this function to print an array of pictures 
    plt.imshow(X_val[i,:,:,:]) # this will call the images from train set one by one
    plt.title('DOG' if y_val[i] == 1 else 'CAT')  # Lets also look into the labels 
    plt.axis('off') 

#3 CNN Model Development and Training

In [36]:
def cnn_model(input_shape):    
    
    X_input = Input(input_shape)

    X = Conv2D(64, (3, 3), strides = (1, 1), padding = 'same', name = 'conv1')(X_input) 
    X = BatchNormalization(axis = 3, name = 'bn1')(X)
    X = Activation('relu')(X) 
    
    X = Conv2D(64, (3, 3), strides = (1, 1), padding = 'same', name = 'conv2')(X_input) 
    X = BatchNormalization(axis = 3, name = 'bn2')(X)
    X = Activation('relu')(X) 

    X = MaxPooling2D((3, 3), name='max_pool_0')(X)
    X = Dropout(0.3)(X)

    X = Conv2D(128, (3, 3), strides = (1, 1), padding = 'same', name = 'conv3')(X) 
    X = BatchNormalization(axis = 3, name = 'bn3')(X)
    X = Activation('relu')(X)
    
    X = Conv2D(128, (3, 3), strides = (1, 1), padding = 'same', name = 'conv4')(X) 
    X = BatchNormalization(axis = 3, name = 'bn4')(X)
    X = Activation('relu')(X)
    
    X = Conv2D(128, (3, 3), strides = (1, 1), padding = 'same', name = 'conv5')(X) 
    X = BatchNormalization(axis = 3, name = 'bn5')(X)
    X = Activation('relu')(X)
   
    X = MaxPooling2D((3, 3), name='max_pool_1')(X)
    X = Dropout(0.3)(X)
    
    # Flatten the data.
    X = Flatten()(X)
    # Dense Layer
    X = Dense(4096, activation='relu', name='fc1')(X)
    X = Dropout(0.1)(X)
    X = Dense(1024, activation='relu', name='fc2')(X)
    X = Dropout(0.1)(X)
    X = Dense(256, activation='relu', name='fc3')(X)
    # Using softmax function to get the output
    X = Dense(1, activation='sigmoid', name='fc4')(X)
    
    model = Model(inputs = X_input, outputs = X, name='model')
    
    return model

In [37]:
cnn_model = cnn_model(X_train.shape[1:4])

In [38]:
from tensorflow.keras.optimizers import Adam
#from keras.optimizers import Adam
epochs = 100
batch_size = 64
learning_rate = 0.001
decay = learning_rate/epochs
optimizer = Adam(learning_rate=learning_rate, epsilon=1e-08, decay = decay)

In [39]:
cnn_model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [40]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
#from keras.callbacks import ReduceLROnPlateau
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=1, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.0001)

In [41]:
from tensorflow.keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=30, verbose=1, restore_best_weights=True, mode='auto')

In [42]:
cnn_history = cnn_model.fit(x = X_train, y = y_train, batch_size = batch_size, 
                        epochs=epochs, verbose=1, 
                        validation_data = (X_val, y_val),
                          shuffle = True, 
                          steps_per_epoch= None, validation_steps=None,
                                  callbacks=[learning_rate_reduction, early_stopping] )

In [43]:
y_train

## 4. Model Performance

In [44]:
preds = cnn_model.evaluate(X_train, y_train)
print ("Loss = " + str(preds[0]))
print ("Train set Accuracy = " + str(preds[1]))

In [45]:
preds_val = cnn_model.evaluate(X_val, y_val)
print ("Loss = " + str(preds_val[0]))
print ("Validation Set Accuracy = " + str(preds_val[1]))

In [46]:
history_df = pd. DataFrame(cnn_history.history)
history_df.loc[:,['loss', 'val_loss']].plot()
history_df.loc[:,['accuracy', 'val_accuracy']].plot()

In [47]:
history_dict = cnn_history.history
history_dict.keys()

In [48]:
val_loss = history_dict['val_loss']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
acc = history_dict['accuracy']
epochs = range(1,len(history_dict['val_loss'])+1)

In [49]:
plt.plot(epochs,acc,'b-')
plt.title('Accuracy of Model')
plt.xlabel('epochs')
plt.ylabel('Accuracy')

plt.plot(epochs,val_acc,'b-', color = 'red')
plt.title('Accuracy of Model')
plt.xlabel('epochs')
plt.ylabel('Accuracy')
plt.show()

In [50]:
plt.plot(epochs,loss,'b-')
plt.title('loss function')
plt.xlabel('epochs')
plt.ylabel('Loss')

plt.plot(epochs,val_loss,'b-', color = 'red')
plt.title('loss function')
plt.xlabel('epochs')
plt.ylabel('val_loss')
plt.show()

##5 Model Prediction and Kaggle submission

In [51]:
predicted_val_probability = cnn_model.predict(X_val, batch_size=64)

In [52]:
y_val_pred_label = np.round(predicted_val_probability)
l = []
for i in range(len(y_val_pred_label)):
    if y_val[i] != y_val_pred_label[i]:
        l.append(i)

In [53]:
m = []
for t in l:
    if predicted_val_probability[t] >= 0.5:
        m.append(t)
    elif predicted_val_probability[t] < 0.5:
        m.append(t)
    

In [54]:
predicted_val_probability[predicted_val_probability >= 0.5] = 1
predicted_val_probability[predicted_val_probability < 0.5] = 0

In [55]:
predicted_val_probability.flatten()

In [56]:
len(predicted_val_probability)

In [70]:
type(predicted_val_probability)

In [69]:
plt.figure(figsize=(20,20))
c = 1
for i in m[:]:
    plt.subplot(5,5,c)
    plt.imshow(X_val[i])
    plt.title('DOG:{}\nTrue label:{}'.format(predicted_val_probability[i], y_val[i])
              if predicted_val_probability[i]>= 0.5 else 'CAT:{}\nTrue label:{}'.format(predicted_val_probability[i],y_val[i]))
    plt.axis('off')
    c = c+1

In [59]:
X_Test_orig = []
for i in os.listdir('/kaggle/working/test/'):
    label = i.split('.')[-2]
    img = cv2.imread('/kaggle/working/test/'+i, cv2.IMREAD_COLOR)
    img = cv2.resize(img,(IMG_SIZE,IMG_SIZE), interpolation = cv2.INTER_CUBIC)
    #img = img / 255
    X_Test_orig.append([np.array(img), np.array(label)])

np.save('Test_Data.npy', X_Train_orig)

In [60]:
X_test = np.array([i[0] for i in X_Test_orig]).reshape(-1,IMG_SIZE, IMG_SIZE, 3)
Label = np.array([i[1] for i in X_Test_orig])

In [63]:
X_test = X_test / 255

In [64]:
classes = cnn_model.predict(X_test, batch_size = batch_size), 
classes

In [65]:
print(max(classes[0]))
print(min(classes[0]))

In [66]:
prediction = pd.DataFrame()
prediction['id'] = Label
prediction['label'] = classes[0].flatten()
prediction['id'] = prediction['id'].astype(int)

In [67]:
type(prediction)
prediction.shape
prediction.info()

In [68]:

#prediction.loc[prediction['label'] >= 0.5, 'label'] = 0.99
#prediction.loc[prediction['label'] < 0.5, 'label'] = 0.01

prediction.sort_values(by='id',ascending=True,inplace=True)
prediction.to_csv('submission_v5.csv', index = False)
prediction.head()